## Metropolis Monte Carlo in Rosetta

@Author: 吴炜坤

@email：weikun.wu@xtalpi.com/weikunwu@163.com

在Rosetta中，具体如何对蛋白质的构象进行采样和优化的呢？这种简短章节将会介绍Metropolis算法的工作方式，以便于初学者理解计算的过程。
如果读者想进一步了解更多细节可以查看本章节下的延展阅读。

### 一、构象的势能面

在之前的章节中，我们知道在给定能量函数的条件下，蛋白质的所有构象状态都有一个对应的能量值，这些连续的能量值共同组成了构象的势能面。天然蛋白质整体的势能面是呈现单个或多个低洼能量阱的漏斗状态，在天然状态下，蛋白质的状态就是全局能量的极小点（们）。

<center><img src="./img/能量面的对比.jpg" width = "900" height = "200" align=center /></center>

**采样和优化问题的本质**: 我们从如何从能量面上随机的一点出发，从而到达全局的能量最低点（天然稳定的状态）？

### 二、Metropolis算法

Metropolis算法是一种蒙特卡洛的采样方法，目的是通过获取一条搜索的轨迹路线来评估蛋白构象的热力学分布的状态的过程。

举个经典的例子: 假设小明可以任意的在地图点上进行“瞬间移动”。目前上级有一个任务，就是测量尼罗河的最深深度的地方。如下图，小明不顾河里的食人鱼，毅然地拿着标尺就开始量。那如何才能让小明有效地测量并完成任务呢？

<center><img src="./img/mcmc_river.jpg" width = "500" height = "200" align=center /></center>

一种最朴素的方法: 随机采样。只要小明的移动速度足够地快，他就可以对整个地图的点进行全局测量，最后获取全局最低的点。但这种方法的采样效率是在太低，因为采用的目标尼罗河仅仅在整个地图版块中占比很小的一个部分。他大部分的精力都用于采样了“无效”的数据点。

接着，小明想出了一种更高效的办法：由于水往低处流，如果我一开始随机选取一个点后，并在之后的移动中，始终沿着附近的低洼地势方向不断前进，那我就有可能找到河流，并不断沿着水底进发。

但这种方式有个致命的问题，整个地图的点上有丘陵，有高低，盆地。如果小明一直往低洼地势前进，就有可能卡限在盆地的区域中。但是如果整体方针是沿着低洼的地区前进，并且以一定的概率向高处逆向行走，那小明就有机会跳出盆地区域，最终走向尼罗河的最深处。

**这种以一定概率接受采样的方式就是Metropolis的基本思想。** 同理，将测量尼罗河最深处的任务替换为搜索蛋白构象势能面最低点处的任务。同样可以以Metropolis算法实现。

### 三、Rosetta中的Metropolis算法

在Rosetta中，具体的Metropolis算法步骤如下:
- 首先初始化一个蛋白质构象i, 如设定随机的骨架二面角;
- 通过一些方法，比如改变蛋白质的二面角，从而得到全新的构象j:
- 计算构象i与构象j之间的Rosetta能量差异$\Delta{E}$;
- 判断是否接受新的构象（Metropolis准则）:
    - 如果$\Delta{E}<=0$, $P_{accept}=1$, 直接接受新的低能量构象.
    - 如果$\Delta{E}>0$, $P_{accept}=\exp \left(-\frac{\Delta{E}}{k T}\right)$, 在[0-1]区间范围内生成随机数u. 如果$u < P_{accept}$接受高能构型，反之拒绝。

<center><img src="./img/Metropolis.jpg" width = "800" height = "200" align=center /></center>

在Rosetta中，Metropolis算法通常与Minimization连用一同，更有效率地去找全局能量的低点。
<center><img src="./img/mcmc_min.jpg" width = "900" height = "200" align=center /></center>